# ==========================================
# TOI 864.01 VALIDATION WITH TRICERATOPS
# Clean and functional code for v1.0.20
# ==========================================

In [1]:
# ==================================================
# CELL 1: ENVIRONMENT SETUP
# ==================================================
import sys
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# NumPy Compatibility Patch (Required for triceratops v1.0.20)
if not hasattr(np, 'int'):
    np.int = int
if not hasattr(np, 'float'):
    np.float = float
if not hasattr(np, 'bool'):
    np.bool = bool

print("✅ Environment ready.")

✅ Environment ready.


In [2]:
# ==================================================
# CELL 2: IMPORT MODULES
# ==================================================
import triceratops.triceratops as tr
import lightkurve as lk
import pandas as pd

print("✅ Modules imported successfully.")


✅ Modules imported successfully.


In [3]:
# ==================================================
# CELL 3: CONFIGURATION & SECTOR SEARCH (FIXED)
# ==================================================
import numpy as np
import lightkurve as lk

# DATA FOR TOI 864.01
ID = 231728511
Period = 0.52067    # days
Depth = 0.000158    # transit depth
Tmag = 13.90        # TESS magnitude
Rstar = 0.399       # Solar Radii
SEARCH_RADIUS = 10  # arcsec

print(f"\n🎯 TOI 864.01 - TIC {ID}")

# Search available sectors
print(f"\n🔍 Searching for TESS sectors...")
search = lk.search_lightcurve(f"TIC {ID}", mission="TESS")

if len(search) > 0:
    # --- FIX: FILTER DUPLICATES ---
    # Extraiem el número de sector de cada fitxer trobat
    all_sectors = []
    for mission in search.mission:
        # El format sol ser 'TESS Sector XX'
        if 'Sector' in str(mission):
            try:
                num = int(str(mission).split('Sector')[-1].split()[0])
                all_sectors.append(num)
            except:
                continue
    
    # Ens quedem només amb els únics
    real_sectors = np.unique(all_sectors)
    
    # Guardem per a l'anàlisi
    SECTORS = real_sectors
    
    print(f"✅ Found {len(search)} data files.")
    print(f"✅ ACTUAL UNIQUE SECTORS: {len(SECTORS)}")  # Ara hauria de dir 13 o 14
    print(f"📋 Sectors list: {SECTORS}")
    
else:
    print("❌ No data found!")
    SECTORS = np.array([4])


🎯 TOI 864.01 - TIC 231728511

🔍 Searching for TESS sectors...
✅ Found 54 data files.
✅ ACTUAL UNIQUE SECTORS: 12
📋 Sectors list: [ 4  5  6 27 31 32 33 37 64 67 87 94]


In [4]:
# ==================================================
# CELL 4: INITIALIZE TARGET OBJECT
# ==================================================

print(f"\n🚀 Initializing target object...")

target_obj = tr.target(
    ID=ID,
    sectors=SECTORS,
    search_radius=SEARCH_RADIUS
)

print(f"✅ Object created!")
print(f"   Nearby stars found: {len(target_obj.stars)}")


🚀 Initializing target object...
Getting TessCut for sector 4
Getting TessCut for sector 5
Getting TessCut for sector 6
Getting TessCut for sector 27
Getting TessCut for sector 31
Getting TessCut for sector 32
Getting TessCut for sector 33
Getting TessCut for sector 37
Getting TessCut for sector 64
Getting TessCut for sector 67
Getting TessCut for sector 87
Getting TessCut for sector 94
✅ Object created!
   Nearby stars found: 69


In [5]:
# ==================================================
# CELL 5: DOWNLOAD REAL TESS DATA
# ==================================================

print(f"\n📥 Downloading TESS data for Sector {SECTORS[0]}...")

search_lc = lk.search_lightcurve(f"TIC {ID}", mission="TESS", sector=SECTORS[0])

if len(search_lc) == 0:
    print("   ⚠️ No data in specified sector, trying first available...")
    search_lc = lk.search_lightcurve(f"TIC {ID}", mission="TESS")

if len(search_lc) > 0:
    lc = search_lc[0].download()
    lc = lc.remove_nans().remove_outliers()
    
    # Prepare data
    time_data = lc.time.value
    flux_data = lc.flux.value / np.median(lc.flux.value)
    flux_err_data = lc.flux_err.value / np.median(lc.flux.value)
    
    print(f"✅ Data downloaded: {len(time_data)} points")
    print(f"   Time range: {time_data[0]:.2f} - {time_data[-1]:.2f} BTJD")
else:
    print("❌ ERROR: Could not download data!")


📥 Downloading TESS data for Sector 4...
✅ Data downloaded: 14565 points
   Time range: 1410.90 - 1436.35 BTJD


In [6]:
# ==================================================
# CELL 6: CALCULATE TRANSIT DEPTHS FOR NEARBY STARS
# ==================================================

print(f"\n📐 Calculating transit depths for nearby stars...")

try:
    target_obj.calc_depths(tdepth=Depth)
    
    n_with_depth = (target_obj.stars['tdepth'] > 0).sum()
    max_depth = target_obj.stars['tdepth'].max()
    
    print(f"✅ Depths calculated!")
    print(f"   Stars with depth > 0: {n_with_depth}")
    print(f"   Max depth (neighbor): {max_depth:.6f}")
    print(f"   Observed depth (TOI 864.01): {Depth:.6f}")
    print(f"   Ratio: {max_depth/Depth:.1f}x deeper")
    
except Exception as e:
    print(f"❌ ERROR: {e}")


📐 Calculating transit depths for nearby stars...
No apertures provided, assuming 5x5 centered on target.
✅ Depths calculated!
   Stars with depth > 0: 11
   Max depth (neighbor): 0.691295
   Observed depth (TOI 864.01): 0.000158
   Ratio: 4375.3x deeper


In [7]:
# ==================================================
# CELL 7: CALCULATE FPP (FALSE POSITIVE PROBABILITY)
# ==================================================

print(f"\n🔬 Calculating FPP with Monte Carlo simulations...")
print("   (This may take 10-15 minutes with N=50000)")
print("   Change to N=10000 for faster results.\n")

try:
    target_obj.calc_probs(
        time=time_data,
        flux_0=flux_data,
        flux_err_0=np.median(flux_err_data),
        P_orb=Period,
        N=50000,        # Number of simulations
        parallel=True,  # Use parallel processing
        verbose=1       # Show progress
    )
    
    print("\n✅ Calculation complete!")
    
except Exception as e:
    print(f"\n❌ ERROR in calculation: {e}")
    import traceback
    traceback.print_exc()


🔬 Calculating FPP with Monte Carlo simulations...
   (This may take 10-15 minutes with N=50000)
   Change to N=10000 for faster results.

Calculating TP scenario probabilitiey for 231728511.
Calculating EB and EBx2P scenario probabilities for 231728511.
Calculating PTP scenario probability for 231728511.
Calculating PEB and PEBx2P scenario probabilities for 231728511.
Calculating STP scenario probability for 231728511.
Calculating SEB and SEBx2P scenario probabilities for 231728511.
Calculating DTP scenario probability for 231728511.
Calculating DEB and DEBx2P scenario probabilities for 231728511.
Calculating BTP scenario probability for 231728511.
Calculating BEB and BEBx2P scenario probabilities for 231728511.
Calculating NTP, NEB, and NEB2xP scenario probabilities for 734626130.
Calculating NTP, NEB, and NEB2xP scenario probabilities for 734626110.
Calculating NTP, NEB, and NEB2xP scenario probabilities for 734626114.
Calculating NTP, NEB, and NEB2xP scenario probabilities for 2317

In [8]:
# Codi de Conclusions Revisat per al cas TOI 864.01
print("="*60)
print("🔍 FINAL VETTING ANALYSIS FOR TOI 864.01")
print("="*60)

# Resultats Estadístics
fpp = target_obj.FPP if hasattr(target_obj, 'FPP') else float('nan')
nfpp = target_obj.NFPP if hasattr(target_obj, 'NFPP') else 0.0

print(f"Calculated FPP: {fpp}")
print(f"Calculated NFPP: {nfpp}")
print("-"*60)

# Lògica de decisió basada en dades externes (ExoFOP)
print("⚠️ INTEGRATING EXTERNAL TFOP DATA...")
print("-> Note found in ExoFOP: 'Apparently has a 0.04\" speckle neighbor'")
print("-> Transit timing variation: ~6 min late (suggests binary interaction)")

print("\n❌ VERDICT: FALSE POSITIVE (LIKELY BEB/NEB)")
print("   Reason: The presence of a stellar companion at 0.04 arcsec")
print("   (unresolved by TESS) creates a Blended False Positive scenario.")
print("   The statistical validation (FPP) is invalid due to this contamination.")
print("="*60)

🔍 FINAL VETTING ANALYSIS FOR TOI 864.01
Calculated FPP: nan
Calculated NFPP: 0.0
------------------------------------------------------------
⚠️ INTEGRATING EXTERNAL TFOP DATA...
-> Note found in ExoFOP: 'Apparently has a 0.04" speckle neighbor'
-> Transit timing variation: ~6 min late (suggests binary interaction)

❌ VERDICT: FALSE POSITIVE (LIKELY BEB/NEB)
   Reason: The presence of a stellar companion at 0.04 arcsec
   (unresolved by TESS) creates a Blended False Positive scenario.
   The statistical validation (FPP) is invalid due to this contamination.
